In [ ]:
import os
import numpy
import matplotlib.pyplot as plt

import clip
import clip.simple_tokenizer
import pickle


In [ ]:
test_output_path = 'save/testset_output'
file_list = os.listdir(test_output_path)
file_list = [f for f in file_list if f.endswith('.pkl')]
file_list = sorted(file_list)
KEYWORD = 'move'
top_tokens = []
for file_name in file_list:
    print(file_name)
    with open(os.path.join(test_output_path, file_name), 'rb') as f:
        batch_output = pickle.load(f)
    bs = batch_output['motion'].shape[0]
    for b in range(bs):
        n_frames = batch_output['lengths'][b]
        attn_value = batch_output['attention_values'][0][b]
        attn_mask = batch_output['attention_masks'][0][b]
        attn_value = attn_value[:,:,attn_mask]
        attn_value = attn_value[:,:,1:(attn_value.shape[-1]-1)]
        attn_value = attn_value/attn_value.sum(axis=-1)[...,None]
        text_input = batch_output['text'][b]
        if KEYWORD not in text_input:
            continue
        text_id = clip.tokenize(text_input, truncate=True)[:,attn_mask][-1].tolist()
        text_token = [clip.clip._tokenizer.decoder[token_id].replace('</w>',"") for token_id in text_id][1:attn_value.shape[-1]+1]

        attn_value = attn_value.mean(axis=0).mean(axis=0)
        top_token_id = numpy.argsort(attn_value)[::-1][:10]
        count = 0 
        for i in top_token_id:
            if text_token[i].lower() not in ['person', 'man', 'someone']:
                if count == 5:
                    break
                count += 1
                top_tokens.append(text_token[i])

In [ ]:
top_token_sentence = ' '.join((top_tokens))
import wordcloud
# create wordcloud object with high dpi, white background, and max words
wc = wordcloud.WordCloud(width=800, height=800, background_color='white', max_words=500, min_font_size=10)
wc.generate(top_token_sentence)
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()


In [ ]:
import clip.simple_tokenizer

num_instances = attn_dict['attn_values'].shape[0]
bs = attn_dict['attn_values'].shape[2]
top_tokens = []

for b in range(bs):
    n_frames = result_dict['lengths'][instance_id]
    attn_value = attn_dict['attn_values'][instance_id][0][b]
    attn_mask = attn_dict['attn_masks'][instance_id][-1][b]
    attn_value = attn_value[:,:,attn_mask]
    attn_value = attn_value[:,:,1:(attn_value.shape[-1]-1)]
    attn_value = attn_value/attn_value.sum(axis=-1)[...,None]

    text_input = result_dict['text'][instance_id*bs+b]
    text_id = clip.tokenize(text_input)[:,attn_mask][0].tolist()
    text_token = [clip.clip._tokenizer.decoder[token_id].replace('</w>',"") for token_id in text_id][1:attn_value.shape[-1]+1]

    attn_value = attn_value.mean(axis=0).mean(axis=0)
    top_token_id = numpy.argsort(attn_value)[::-1][:10]
    count = 0 
    for i in top_token_id:
        if text_token[i].lower() not in ['person', 'man', 'someone']:
            if count == 5:
                break
            count += 1
            top_tokens.append(text_token[i])

        
top_token_sentence = ' '.join(top_tokens)
import wordcloud
# create wordcloud object with high dpi, white background, and max words
wc = wordcloud.WordCloud(width=800, height=800, background_color='white', max_words=200, min_font_size=10)
wc.generate(top_token_sentence)
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

        

## Plot for single batch

In [16]:
pickle_path = 'save/testset_output/MDM_CLIP_ATTN_batch_1.pkl'

with open(pickle_path, 'rb') as f:
    outputs = pickle.load(f)
    
result_dict = outputs
attn_dict = outputs

In [ ]:
outputs.keys()

In [ ]:
import clip.simple_tokenizer

num_instances = attn_dict['attention_values'].shape[0]
bs = attn_dict['attention_values'].shape[1]


for b in range(bs):
    n_frames = result_dict['lengths'][b]
    attn_value = attn_dict['attention_values'].mean(axis=0)[b]
    attn_mask = attn_dict['attention_masks'][-1][b]
    attn_value = attn_value[:,:,attn_mask]
    attn_value = attn_value[:,:,1:(attn_value.shape[-1]-1)]
    attn_value = attn_value/attn_value.sum(axis=-1)[...,None]

    text_input = result_dict['text'][b]
    text_id = clip.tokenize(text_input)[:,attn_mask][0].tolist()
    text_token = [clip.clip._tokenizer.decoder[token_id].replace('</w>',"") for token_id in text_id][1:attn_value.shape[-1]+1]

    # plot attention by head
    num_head = attn_value.shape[0]
    fig, axs = plt.subplots(1, num_head+1, figsize=(25, 10))
    for h in range(num_head):
        axs[h].imshow(attn_value[h].T, cmap='hot', interpolation='nearest')
        # use the text_token as x-axis
        axs[h].set_yticks(numpy.arange(len(text_token)))
        axs[h].set_yticklabels(text_token, rotation=0)
        # make each subplot wider
        axs[h].set_aspect('auto')
        axs[h].set_title(f'head {h}')
    # plot the average attention
    axs[num_head].imshow(attn_value.mean(axis=0).T, cmap='hot', interpolation='nearest')
    axs[num_head].set_yticks(numpy.arange(len(text_token)))
    axs[num_head].set_yticklabels(text_token, rotation=0)
    axs[num_head].set_aspect('auto')
    axs[num_head].set_title(f'average')
    
    plt.show()
    # save_path = os.path.join(path, f'attn_rep{instance_id}_b{b}.png')
    # fig.savefig(save_path)

## Analysis for individual prompt

In [ ]:
path = 'save/Best_MDM_V2/samples_Best_MDM_V2_000100000_seed10_a_person_wave_his_arm,_and_then_sit_down'

result_path = os.path.join(path, 'results.npy')
attn_path = os.path.join(path, 'results_attn.npy')

result_dict = numpy.load(result_path, allow_pickle=True).item()
attn_dict = numpy.load(attn_path, allow_pickle=True).item()

In [ ]:
# plot attention for key frames [0, 20, 40, 60, 80, 100]
# x axis: text token
# y axis: head 1, 2, 3, 4, average

num_instances = attn_dict['attn_values'].shape[0]
bs = attn_dict['attn_values'].shape[2]

key_frames = [0, 20, 40, 60, 80, 100]

for instance_id in range(num_instances):
    for b in range(bs):
        n_frames = result_dict['lengths'][instance_id]
        attn_value = attn_dict['attn_values'][instance_id][0][b]
        attn_mask = attn_dict['attn_masks'][instance_id][-1][b]
        attn_value = attn_value[:,:,attn_mask]
        attn_value = attn_value[:,:,1:(attn_value.shape[-1]-1)] # num_head x num_frame x num_token
        attn_value = attn_value/attn_value.sum(axis=-1)[...,None]


        text_input = result_dict['text'][instance_id*bs+b]
        text_id = clip.tokenize(text_input)[:,attn_mask][0].tolist()
        text_token = [clip.clip._tokenizer.decoder[token_id].replace('</w>',"") for token_id in text_id][1:attn_value.shape[-1]+1]

        # plot attention by frames
        attn_value = numpy.concatenate([attn_value, attn_value.mean(axis=0)[None]], axis=0)
        fig, axs = plt.subplots(1, len(key_frames), figsize=(45, 5))
        for i, frame in enumerate(key_frames):
            axs[i].imshow(attn_value[:,frame,], cmap='hot_r', interpolation='nearest')
            axs[i].set_xticks(numpy.arange(len(text_token)))
            axs[i].set_xticklabels(text_token, rotation=45, fontsize=20)
            axs[i].set_yticks(numpy.arange(attn_value.shape[0]))
            if i == 0:
                axs[i].set_yticklabels(['Head 1', 'Head 2', 'Head 3', 'Head 4', 'Average'], rotation=0, fontsize=15)
            # axs[i].set_yticklabels(['Head 1', 'Head 2', 'Head 3', 'Head 4', 'Average'], rotation=0, fontsize=15)
            axs[i].set_aspect('auto')
            axs[i].set_title(f'Frame {frame}', fontsize=20)
        plt.show()
        

In [ ]:
import clip.simple_tokenizer

num_instances = attn_dict['attn_values'].shape[0]
bs = attn_dict['attn_values'].shape[2]

# instance_id = 0
for instance_id in range(num_instances):
    for b in range(bs):
        n_frames = result_dict['lengths'][instance_id]
        attn_value = attn_dict['attn_values'][instance_id][0][b]
        attn_mask = attn_dict['attn_masks'][instance_id][-1][b]
        attn_value = attn_value[:,:,attn_mask]
        attn_value = attn_value[:,:,1:(attn_value.shape[-1]-1)]
        attn_value = attn_value/attn_value.sum(axis=-1)[...,None]

        text_input = result_dict['text'][instance_id*bs+b]
        text_id = clip.tokenize(text_input)[:,attn_mask][0].tolist()
        text_token = [clip.clip._tokenizer.decoder[token_id].replace('</w>',"") for token_id in text_id][1:attn_value.shape[-1]+1]

        # plot attention by head
        num_head = attn_value.shape[0]
        fig, axs = plt.subplots(1, num_head+1, figsize=(25, 5))
        for h in range(num_head):
            axs[h].imshow(attn_value[h], cmap='hot', interpolation='nearest')
            # use the text_token as x-axis
            axs[h].set_xticks(numpy.arange(len(text_token)))
            axs[h].set_xticklabels(text_token, rotation=45)
            # y-axis is the frame index (every 20 frames)
            axs[h].set_yticklabels(numpy.arange(-20, n_frames, 20))
            # make each subplot wider
            axs[h].set_aspect('auto')
            axs[h].set_title(f'head {h}')
        # plot the average attention across heads
        axs[num_head].imshow(attn_value.mean(axis=0), cmap='hot', interpolation='nearest')
        axs[num_head].set_xticks(numpy.arange(len(text_token)))
        axs[num_head].set_xticklabels(text_token, rotation=45)
        # y-axis is the frame index (every 20 frames)
        axs[num_head].set_yticklabels(numpy.arange(-20, n_frames, 20))
        axs[num_head].set_aspect('auto')
        axs[num_head].set_title(f'average')       
        
        plt.show()
        save_path = os.path.join(path, f'attn_rep{instance_id}_b{b}.png')
        fig.savefig(save_path)

In [ ]:
import clip.simple_tokenizer

num_instances = attn_dict['attn_values'].shape[0]
bs = attn_dict['attn_values'].shape[2]
top_tokens = []

# instance_id = 0
for instance_id in range(num_instances):
    for b in range(bs):
        n_frames = result_dict['lengths'][instance_id]
        attn_value = attn_dict['attn_values'][instance_id][0][b]
        attn_mask = attn_dict['attn_masks'][instance_id][-1][b]
        attn_value = attn_value[:,:,attn_mask]
        attn_value = attn_value[:,:,1:(attn_value.shape[-1]-1)]
        attn_value = attn_value/attn_value.sum(axis=-1)[...,None]

        text_input = result_dict['text'][instance_id*bs+b]
        text_id = clip.tokenize(text_input)[:,attn_mask][0].tolist()
        text_token = [clip.clip._tokenizer.decoder[token_id].replace('</w>',"") for token_id in text_id][1:attn_value.shape[-1]+1]

        attn_value = attn_value.mean(axis=0).mean(axis=0)
        top_token_id = numpy.argsort(attn_value)[::-1][:10]
        count = 0 
        for i in top_token_id:
            if text_token[i].lower() not in ['person', 'man', 'someone']:
                if count == 5:
                    break
                count += 1
                top_tokens.append(text_token[i])

        
top_token_sentence = ' '.join(top_tokens)
import wordcloud
# create wordcloud object with high dpi, white background, and max words
wc = wordcloud.WordCloud(width=800, height=800, background_color='white', max_words=200, min_font_size=10)
wc.generate(top_token_sentence)
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

        